Import required libraries.

In [194]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [195]:
data = pd.read_csv('datasets/Churn.csv')

In [196]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [198]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Feature Preparation

I will fill in the missing values in `Tenure` with 0, because these customers may not have fixed deposits thus the period of maturation cannot be determined.

In [199]:
data['Tenure'].fillna(0, inplace=True)

In [200]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.543400,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,3.111573,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,4.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


For feature preparation, I will use Ordinal Encoding to encode textual categories with numbers.

I will use the improrted OrdinalEncoder from sklearn.preprocessing.

In [201]:
encoder = OrdinalEncoder() 
data = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

In [202]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.0,2736.0,1115.0,228.0,0.0,0.0,24.0,2.0,0.0,0.0,1.0,1.0,5068.0,1.0
1,1.0,3258.0,1177.0,217.0,2.0,0.0,23.0,1.0,743.0,0.0,0.0,1.0,5639.0,0.0
2,2.0,2104.0,2040.0,111.0,0.0,0.0,24.0,8.0,5793.0,2.0,1.0,0.0,5707.0,1.0
3,3.0,5435.0,289.0,308.0,0.0,0.0,21.0,1.0,0.0,1.0,0.0,0.0,4704.0,0.0
4,4.0,6899.0,1822.0,459.0,2.0,0.0,25.0,2.0,3696.0,0.0,1.0,1.0,3925.0,0.0


Split the data into training, validation and test set, in 60-20-20 proportions, using the test_train_split function.

In [203]:
features = data.drop('Exited', axis=1)
target = data['Exited']

In [204]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, 
                                                                              test_size=0.25, random_state=12345) 

### Balance of the classes:

For a class to be balanced, the accuracy of the class should be equal to the accuracy of the constant model.

I will create a constant model that predicts 0 for any observation.
Then I will find the accuracy of the constant model.

In [205]:
target_pred_constant = pd.Series(0, index=target.index)

print(accuracy_score(target, target_pred_constant))

0.7963


**Since the accuracy is far from 1, this means the classes are imbalanced, and this imbalance will affect how the model is trained.**

1. Decision Tree Classifier 

In [206]:
#a loop for max_depth from 1 to 5
for i in range(1,11):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model_dt.fit(features_train, target_train)
    dt_valid_predictions = model_dt.predict(features_valid)
    f1_score_dt = f1_score(target_valid, dt_valid_predictions)
    print('max_depth = ', i , ':', f1_score_dt)

max_depth =  1 : 0.0
max_depth =  2 : 0.5037037037037037
max_depth =  3 : 0.39382239382239387
max_depth =  4 : 0.4336448598130841
max_depth =  5 : 0.5238879736408567
max_depth =  6 : 0.5067114093959731
max_depth =  7 : 0.5213270142180094
max_depth =  8 : 0.5421133231240428
max_depth =  9 : 0.5407854984894259
max_depth =  10 : 0.5198237885462555


**The highest F1 score of 54.2%, is achieved at a max_depth of 8.**

2. Random Forest Classifier

In [207]:
#a loop for number of estimators from 1 to 10:
for i in range(10, 31):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=i)
    model_rf.fit(features_train, target_train)
    rf_valid_predictions = model_rf.predict(features_valid)
    f1_rf = f1_score(target_valid, rf_valid_predictions)
    print(i, f1_rf)

10 0.5245346869712352
11 0.5252525252525253
12 0.5357737104825291
13 0.53
14 0.54
15 0.5346534653465347
16 0.5370675453047776
17 0.5436573311367381
18 0.5385878489326765
19 0.5448613376835236
20 0.5460526315789473
21 0.5490196078431372
22 0.5304777594728172
23 0.5304777594728172
24 0.5249169435215947
25 0.533112582781457
26 0.5298013245033113
27 0.5337726523887973
28 0.5315614617940199
29 0.5337726523887973
30 0.5361842105263158


**The highest F1 score is 55% at 100 trees and a depth of 21.**

In [208]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear')
model_lr.fit(features_train, target_train)
lr_valid_predictions = model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, lr_valid_predictions)
print(f1_lr)

0.29411764705882354


**Logistic Regression has the least F1 score of 29%.**

### Fixing the class imbalance to improve model quality
1. Upsampling:
    - increasing the sample size
    - increases thenumber ofobservations by duplicating the rarer class observations several times.
    
The `upsample` function will:
- Split the training sample by class
- Determine the class with fewer observations 
- Duplicate the rarer class observations several times
- Create a new training sample based on the data obtained
- Shuffle the data


In [209]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

In [210]:
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=8, class_weight='balanced')
model_dt.fit(features_upsampled, target_upsampled)
dt_valid_predictions = model_dt.predict(features_valid)
f1_score_dt = f1_score(target_valid, dt_valid_predictions)
print(f1_score_dt)

0.5408062930186824


In [211]:
model_rf = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=12, class_weight='balanced')
model_rf.fit(features_upsampled, target_upsampled)
predicted_valid = model_rf.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5933250927070457


In [212]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_lr.fit(features_upsampled, target_upsampled)
lr_valid_predictions = model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, lr_valid_predictions)
print(f1_lr)

0.4722222222222222


### The F1 score of the models have improved significantly after balancing the classes and upsampling.

2. Downsampling:
    - decreases the number of observations by randomly dropping the majority class observations.
    
The `downsample` function will perform the same as the `upsample`. The only difference is that instead of duplicating the rarer observations, `downsample` will randomly drop a fraction of the majority class observations.

In [213]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

In [214]:
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=8, class_weight='balanced')
model_dt.fit(features_downsampled, target_downsampled)
dt_valid_predictions = model_dt.predict(features_valid)
f1_score_dt = f1_score(target_valid, dt_valid_predictions)
print(f1_score_dt)

0.4955752212389381


In [215]:
model_rf = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=12, class_weight='balanced')
model_rf.fit(features_downsampled, target_downsampled)
predicted_valid = model_rf.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.45825242718446607


In [216]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model_lr.fit(features_downsampled, target_downsampled)
lr_valid_predictions = model_lr.predict(features_valid)
f1_lr = f1_score(target_valid, lr_valid_predictions)
print(f1_lr)

0.4703862660944206


### The balanced and downsampled F1 score has improved, in comparison to the imbalanced models, but it is lower than the upsampled balanced models.

### Testing the selected model.

In [217]:
model_rf = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=4, class_weight='balanced')
model_rf.fit(features_train, target_train)
predicted_test = model_rf.predict(features_test)
f1_test = f1_score(target_test, predicted_test)
print(f1_test)

0.5940409683426443


### Calculating the AU-ROC

AUC-ROC is the area under the curve of the ROC. It is an evaluation metric with values in the range from 0 to 1.

The higher the ROC curve, the greater the TPR value and the better the model's quality.
   - Therefore, the higher the AUC-ROC, the better the model's quality.

In [218]:
probabilities_test = model_rf.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(auc_roc)

0.8479970104411237


**F1 is 0.59, while AUC-ROC is 0.84**

**Since F1 is the harmonic mean of *recall* and *precision* , and it is closer to 1, this means that the quality of the model in identifying true positives is good.**

**AUC-ROC is also closer to 1. This means that the model has better quality.**

**The balanced Random Forest Classifier with 100 trees and depth of 4 can be used to predict whether a customer will leave the bank soon, because of it's good quality.**